In [1]:
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
from PIL import Image
import numpy as np
import cv2
from pytesseract import Output
import pandas as pd
from pdfminer.high_level import extract_text
import re

In [2]:
# Loading Labeled Dataset
# x_train, y_train = load_labeled_data()

In [2]:
img_path = r"C:\Users\iamde\GitHub\ML-Basics\TEXT Extraction Using AI\Edominer-1201_page-1.jpg"
annotations_path = r"C:\Users\iamde\GitHub\ML-Basics\TEXT Extraction Using AI\Edominer-1201_page-1.xml"

In [3]:
def load_image(img_path):
    img = cv2.imread(img_path)
    
    # Check if image was loaded successfully
    if img is None:
        print(f"Error: Could not load image at path: {img_path}")
        return FileNotFoundError  # raise an exception 
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

image = load_image(img_path)

# Check if image loading was successful before proceeding
if image is not None:
    print("Image Shape:",image.shape)
else:
    print("Image loading failed. Exiting.")

Image Shape: (3505, 2480, 3)


In [4]:
print("Image Shape:",image.shape)

Image Shape: (3505, 2480, 3)


In [5]:
cv2.imshow('Image', image)
cv2.waitKey(0)

-1

In [6]:
import os
from lxml import etree

def load_annotations(xml_path):
    tree = etree.parse(xml_path)
    root = tree.getroot()
    bboxes = []
    for obj in root.findall('object'):
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)
        bboxes.append([xmin, ymin, xmax, ymax])
    return bboxes



bboxes = load_annotations(annotations_path)

In [7]:
print("Bounding Boxes:",bboxes)

Bounding Boxes: [[140, 375, 1405, 472], [138, 469, 886, 518], [140, 518, 894, 567], [421, 640, 1086, 704], [419, 704, 1081, 848], [419, 856, 1094, 904], [419, 904, 1100, 956], [419, 953, 1092, 1002], [416, 999, 1094, 1056], [413, 1050, 1094, 1113], [135, 631, 424, 1107], [1097, 642, 1554, 737], [1562, 637, 2038, 742], [148, 1177, 1097, 1237], [148, 1231, 1089, 1377], [148, 1380, 1089, 1431], [146, 1429, 711, 1485], [719, 1429, 1081, 1485], [148, 1483, 1081, 1540], [154, 1685, 2038, 1753], [184, 1745, 1146, 3213]]


In [8]:
#training data
x_train = np.array([image])
y_train = np.array(bboxes)

print(x_train.shape)
print(y_train.shape)

(1, 3505, 2480, 3)
(21, 4)


In [10]:
x_train

array([[[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        ...,

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],

        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
        

In [9]:
y_train

array([[ 140,  375, 1405,  472],
       [ 138,  469,  886,  518],
       [ 140,  518,  894,  567],
       [ 421,  640, 1086,  704],
       [ 419,  704, 1081,  848],
       [ 419,  856, 1094,  904],
       [ 419,  904, 1100,  956],
       [ 419,  953, 1092, 1002],
       [ 416,  999, 1094, 1056],
       [ 413, 1050, 1094, 1113],
       [ 135,  631,  424, 1107],
       [1097,  642, 1554,  737],
       [1562,  637, 2038,  742],
       [ 148, 1177, 1097, 1237],
       [ 148, 1231, 1089, 1377],
       [ 148, 1380, 1089, 1431],
       [ 146, 1429,  711, 1485],
       [ 719, 1429, 1081, 1485],
       [ 148, 1483, 1081, 1540],
       [ 154, 1685, 2038, 1753],
       [ 184, 1745, 1146, 3213]])

In [ ]:
model =Sequential()
model.add(Embedding(10000,128))
model.add(LSTM(128))
model.add(Dense(20,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',matrices=['accuracy'])
model.fit(x_train, y_train, epochs=5)

print("Model training complete.")


In [9]:
import numpy as np

# If you have a single image, expand its dimension to match batch size
images = np.expand_dims(image, axis=0)  # Now the shape is (1, 3505, 2480, 3)


In [10]:
import tensorflow as tf

# Annotations is a NumPy array of shape (20, 4)
annotations = np.array(bboxes)

# Create a dataset from images and annotations
dataset = tf.data.Dataset.from_tensor_slices((images.repeat(len(annotations), axis=0), annotations))

# Shuffle and batch the dataset
dataset = dataset.shuffle(buffer_size=20).batch(32).prefetch(tf.data.experimental.AUTOTUNE)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(3505, 2480, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    
    # Output for 4 coordinates of bounding box
    tf.keras.layers.Dense(4)  
])

model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

model.fit(dataset, epochs=10)
print("Model training complete.")


# Solution of Out of Memory Error

In [ ]:
def resize_image(image, target_size=(256, 256)):
    return cv2.resize(image, target_size)

# image_path = 'path/to/your/image.jpg'
image = cv2.imread(img_path)
image_resized = resize_image(image)
image_resized.shape

In [13]:
def load_and_preprocess_image(path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image)
    image = image / 255.0
    return 

In [ ]:
def resize_image(image, target_size=(256, 256)):
    return cv2.resize(image, target_size)

image_resized = resize_image(image)
image_resized.shape

In [ ]:

def load_and_preprocess_image(path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = resize_image(image)
    image = image / 255.0
    return image

def load_and_preprocess_from_path_label(path, xmin, ymin, xmax, ymax, label):
    return load_and_preprocess_image(path), (xmin, ymin, xmax, ymax, label)

paths = ['path/to/image1.jpg', 'path/to/image2.jpg']  # Example paths
xmins = [100, 200]
ymins = [100, 200]
xmaxs = [150, 250]
ymaxs = [150, 250]
labels = [1, 0]

dataset = tf.data.Dataset.from_tensor_slices((paths, xmins, ymins, xmaxs, ymaxs, labels))
dataset = dataset.map(load_and_preprocess_from_path_label)
dataset = dataset.shuffle(buffer_size=1000).batch(4).prefetch(tf.data.experimental.AUTOTUNE)


In [16]:
input_shape = (256,256,3)
# Simple Model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=input_shape),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4)
])


In [ ]:
tf.keras.mixed_precision.set_global_policy('mixed_float16')


In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

model.fit(dataset, epochs=10)
print("Model training complete.")